In [ ]:
# BEFORE RUNNING UPLOAD ALL DATA IN nptel_face_recognition  FOLDER ON DRIVE

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.27 ms


In [ ]:
try:
    import face_recognition
except:
    !pip install face_recognition
    import face_recognition

time: 2.49 ms


In [ ]:
import numpy as np
import time
import requests
import os
from pathlib import Path
import pickle
from lxml import html
import numpy as np
import urllib.request
import cv2
import matplotlib.pyplot as plt
from googlesearch import search
from PIL import Image, ImageStat
import torch
import pandas as pd

import time
tt = time.time
st = tt()

time: 7.9 ms


In [ ]:
# mount drive 
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.81 ms


In [ ]:
# for colab
colab = True
base_dir = "/content/drive/My Drive/nptel_face_recognition"
print(f"os.getcwd(): {os.getcwd()}")
if not os.getcwd() == base_dir:
    os.chdir(base_dir)
    print(f"os.getcwd(): {os.getcwd()}")

os.getcwd(): /content/drive/My Drive/nptel_face_recognition
time: 6.59 ms


In [ ]:
# get prof name from thumbnail
def save_as_pickle(file_path, file):
    with open(file_path, 'wb') as f:
        pickle.dump(file, f)

def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        file = pickle.load(f)
    return file

def plot_image_list(img_list, figsize=(12, 12), subplot_n_cols=2):
    plt.figure(figsize=(12, 12))
    for i,img in enumerate(img_list):
        plt.subplot(int(len(img_list)/subplot_n_cols)+1, subplot_n_cols, i+1)
        try:
            plt.imshow(img)
        except:
            pass
        
def brightness(img):
    img = Image.fromarray(img)
    img = img.convert('L')
    stat = ImageStat.Stat(img)
    return stat.rms[0]

def is_valid_img(img, lthresh = 100, hthresh = 200):
    bright = brightness(img)
    print(bright)
    if bright<lthresh or bright>hthresh:
        return False
    else:
        return True
        
def get_extension_files(path_to_json, ext='.json'):
    json_files = [os.path.join(path_to_json, pos_json) for pos_json in os.listdir(path_to_json) if pos_json.endswith(ext)]
    return json_files

def flatten_list(l):
    return [item for sublist in l for item in sublist]

def post_processing(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm_img = cv2.normalize(img, norm_img, 0, 255,
                             cv2.NORM_MINMAX)
    return img

def get_unique_counts_from_array(arr):
    unique, counts = np.unique(np.array(arr), return_counts=True)
    return unique, counts

def get_unique_counts_from_unique_pixel_values(img):
    uniq_img = img[:,:,0] + 255*img[:,:,1] + 255*255*img[:,:,1]
    uniq_img = uniq_img.flatten()  
    return get_unique_counts_from_array(uniq_img)

def get_per_most_common_pixels(img):
    unique, counts = get_unique_counts_from_unique_pixel_values(img)
    counts = list(np.sort(counts))
    highest_cnt = counts[0]
    return len([w for w in counts if w==highest_cnt])/len(counts)
    
def is_slide(img, threshold=0.5):
    score = get_per_most_common_pixels(img)
    if score>threshold:
        return True
    else:
        return False

print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")
fr_fl = face_recognition.face_locations
fr_fe = face_recognition.face_encodings

if torch.cuda.is_available():
    fr_fl_dict = {"number_of_times_to_upsample":3, "model":"cnn"}
    fr_fe_dict = {"num_jitters":3, "model":"large"}  
else:
    fr_fl_dict = {"number_of_times_to_upsample":1, "model":"hog"}
    fr_fe_dict = {"num_jitters":1, "model":"small"}  



torch.cuda.is_available(): True
time: 63.6 ms


In [ ]:
data_dir = os.path.join(base_dir, 'data')
results_dir = os.path.join(base_dir, "results")
os.makedirs(results_dir, exist_ok = True)

# ver=""
ver="_with_diversity"
known_faces_dir = os.path.join(data_dir, 'known_faces'+ver)
full_vedios_dir = os.path.join(data_dir, 'full_vedio')
pkl_files_dir = os.path.join(data_dir, "pkl_files")
csv_files_dir = os.path.join(data_dir, "csv_files")

folder_name_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict"+ver+".pkl"))

images_dir_list = [x[0] for x in os.walk(known_faces_dir)][1:]
known_face_names = [folder_name_2_prof_name_dict[int(w.split('/')[-1])] for w in images_dir_list]

known_face_images_nested_list = []
for image_dir in images_dir_list:
    known_images_path_list = get_extension_files(image_dir, ext='.jpg') 
    known_face_images_nested_list.append([cv2.imread(w)[:, :, ::-1] for w in known_images_path_list])
#     known_face_images_nested_list.append([face_recognition.load_image_file(w) for w in known_images_path_list])

filter_idx_list=[]
known_face_encodings=[]
known_face_seprate_encodings=[]
for i,(n,nl) in enumerate(zip(known_face_names, known_face_images_nested_list)):
    enc_list = []
    for img in nl:
        # making frame smaller to try to solve cuda malloc error
        img = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
        known_face_locations = fr_fl(img, **fr_fl_dict)
        enc = fr_fe(img, known_face_locations = known_face_locations, **fr_fe_dict)
        if enc!=[]:
            enc_list.append(enc[0])
    
    seprate_img_arr = np.array(enc_list)
    img_arr = np.array(enc_list).mean(axis=0)
    if enc_list!=[]:
        known_face_encodings.append(img_arr)
        known_face_seprate_encodings.append(seprate_img_arr)
    else:
        print(f"encoding not found for {n}")
        filter_idx_list.append(i)

known_face_names = [known_face_names[i] for i in range(len(known_face_names)) if i not in filter_idx_list]    

time: 31.5 s


In [ ]:
save_as_pickle(os.path.join(pkl_files_dir, "known_face_names"+ver+".pkl"), known_face_names)
save_as_pickle(os.path.join(pkl_files_dir, "known_face_encodings"+ver+".pkl"), known_face_encodings)
save_as_pickle(os.path.join(pkl_files_dir, "known_face_seprate_encodings"+ver+".pkl"), known_face_seprate_encodings)

time: 27.8 ms


In [ ]:
tracebacks_dir = os.path.join(data_dir, "tracebacks")
pkl_files_dir = os.path.join(data_dir, "pkl_files")
os.makedirs(tracebacks_dir, exist_ok = True)

nptel_channel_name_list = load_from_pickle(os.path.join(pkl_files_dir, "nptel_channel_name_list.pkl"))
playlist_2_prof_name_dict = load_from_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"))

# known_face_names = load_from_pickle(os.path.join(pkl_files_dir, "known_face_names.pkl"))
# known_face_encodings = load_from_pickle(os.path.join(pkl_files_dir, "known_face_encodings.pkl"))

# start after labeled faces
# TODO: custom vedio start idx as all vedios dont take images from the first 2 vedios
vedio_start_idx = 2
# treshold for face distance
threshold = 0.25
max_pred_cnt = 50
threshold_list = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

num_channels = 5
num_playlists = 5
num_vedios = 5

results_dict_list = []
# for nptel_channel_name in nptel_channel_name_list:
for nptel_channel_name in nptel_channel_name_list[:num_channels]:
    print(f'\n\n\nfor nptel_channel_name {nptel_channel_name}')
    chanel_pkl_files_dir = os.path.join(pkl_files_dir, nptel_channel_name)
    facef_playlist_names_list = load_from_pickle(os.path.join(chanel_pkl_files_dir, "facef_playlist_names_list.pkl"))

    results_chanel_dir = os.path.join(results_dir, nptel_channel_name)
    chanel_vedio_dir = os.path.join(full_vedios_dir, nptel_channel_name)
    paylist_path_list = [x[0] for x in os.walk(chanel_vedio_dir)][1:]
    paylist_name_list = [x.split('/')[-1] for x in paylist_path_list] 
    
    vedio_ext = '.mp4'
    
    namef=0
    namenf=0
    namef_playlist_names_list=[]
    namenf_playlist_names_list=[]
    # for playlist, playlistn in zip(paylist_path_list, paylist_name_list):
    for playlist, playlistn in zip(paylist_path_list[:num_playlists], paylist_name_list[:num_playlists]):
#         if playlistn not in facef_playlist_names_list:
#             continue
        print(f"\n\nplaylist: {playlistn}")
        true_name = playlist_2_prof_name_dict[playlistn]
        if true_name is None:
            print(f"no name for {playlistn}")
            continue
            
        if true_name not in folder_name_2_prof_name_dict.values():
            print(f"no face for {true_name}")
            continue
        
        results_playlist_dir = os.path.join(results_chanel_dir, playlistn)
        
        vedio_path_list = get_extension_files(playlist, ext='.mp4')
        vedio_name_list = [x.split('/')[-1] for x in vedio_path_list] 
        
        
        for vedio_path in vedio_path_list[vedio_start_idx:vedio_start_idx+num_vedios]:
            vname = vedio_path.split('/')[-1]

            video_capture = cv2.VideoCapture(vedio_path)

            # Initialize some variables
            face_locations = []
            face_encodings = []
            face_names = []
            process_this_frame = True

            vst = 0
            vet = 3*60
            num_frames_per_sec = 1

            fps = int(video_capture.get(cv2.CAP_PROP_FPS))
            st = tt()
            i=-1
            pred_cnt = 0
            y_pred=[]
            names_list = []
            frame_list = []
            frame_index_list=[]
            slides_index_list_list = [[]]*len(threshold_list)
            names_scores_list = []
            results_dict = {}
            while video_capture.isOpened():
                i+=1
                if int(i/fps) < vst:
                    ret, frame = video_capture.read()
                    continue

                if int(i/fps) > vet:
                    break

                 # Grab a single frame of video
                ret, frame = video_capture.read()

                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

                # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
                rgb_frame = small_frame[:, :, ::-1]

                # process
                if i%int(fps/num_frames_per_sec)==0:
                    for j, threshold in enumerate(threshold_list):
                        if is_slide(frame, threshold = threshold):
                            slides_index_list_list[j].append(int(i/fps))

                    face_locations = fr_fl(rgb_frame, **fr_fl_dict)
                    if not len(face_locations)==1:
                        continue

                    pred_cnt+=1
                    if pred_cnt>max_pred_cnt:
                        break
                
                    face_encoding = fr_fe(rgb_frame, known_face_locations = face_locations, **fr_fe_dict)[0]

                    # face distances with average encodings
                    face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)

                    # face distances as sum of distances for all faces of a prof
#                     face_distances=[]
#                     for known_face_encodings_list in known_face_seprate_encodings:
#                         face_distances.append(face_recognition.face_distance(known_face_encodings_list, face_encoding).mean())

                    best_match_index = np.argmin(face_distances)
                    name = known_face_names[best_match_index]
        
#                     if face_distances[best_match_index]<threshold:
#                         name = 'unknown'
#                     else:
#                         name = known_face_names[best_match_index]
                    
#                     print(f"true_name: {true_name}")
#                     print(f"name: {name}")
#                     print(f"face_distances[best_match_index]: {face_distances[best_match_index]}")
#                     print(f"time: {int(i/fps)}")
                    
                    names_list.append(name)
                    frame_list.append(frame)
                    frame_index_list.append(int(i/fps))
                    names_scores_list.append((name, face_distances[best_match_index]))
            
                    # results_vedio_dir = os.path.join(results_playlist_dir, vname)
                    # face_recognition_results_dir = os.path.join(results_vedio_dir, "face_recogition")
                    # tp_dir = os.path.join(face_recognition_results_dir, "tp")
                    # fp_dir = os.path.join(face_recognition_results_dir, "fp")
                    # os.makedirs(tp_dir, exist_ok = True)
                    # os.makedirs(fp_dir, exist_ok = True)

                    if name==true_name:
                        y_pred.append(1)
#                         image_path = os.path.join(tp_dir, name+'.jpg')
#                         cv2.imwrite(image_path, cv2.cvtColor(rgb_small_frame, cv2.COLOR_RGB2BGR))
                    else:
                        y_pred.append(0)
#                         image_path = os.path.join(fp_dir, name+'.jpg')
#                         cv2.imwrite(image_path, cv2.cvtColor(rgb_small_frame, cv2.COLOR_RGB2BGR))

                # Hit 'q' on the keyboard to quit!
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            # Release handle to the webcam
            video_capture.release()
            cv2.destroyAllWindows()
            
            # save_as_pickle(os.path.join(results_vedio_dir, "names_scores_list"+ver+".pkl"), names_scores_list)
                
            print(f"\nvedio: {vname}")
            print("unique predicted value counts:")
            unique, counts = np.unique(np.array(names_list), return_counts=True)
            print(np.asarray((unique, counts)).T)
            print(f"true_name: {true_name} \n accuracy:{np.array(y_pred).mean()} \
            \nnumber of predictions: {len(y_pred)}\ntime: {tt()-st}")
            results_dict = {"vname": vname,
                            "nptel_channel_name": nptel_channel_name, 
                            "playlistn": playlistn,
                            "vedio_path": vedio_path,
                            "true_name": true_name,
                            "accuracy": np.array(y_pred).mean(),
                            "number of predictions": len(y_pred),
                            "time": tt()-st,
                            "unique": unique,
                            "counts": counts,
                            "frame_index_list":frame_index_list,
                            "names_list":names_list,
                            }
            for j, threshold in enumerate(threshold_list):
                results_dict[f"slides_index_list_{threshold}"] = slides_index_list_list[j]
            results_dict_list.append(results_dict)

results_dict_csv = pd.DataFrame(results_dict_list)
results_dict_csv.to_csv(os.path.join(csv_files_dir, "face_recognition_results"+ver+".csv"))




for nptel_channel_name Aerospace Engineering


playlist: Jet and Rocket Propulsion

vedio: ZDQBcuVRWYs.mp4
unique predicted value counts:
[['dr. a. kushari' '45']
 ['dr. p. gopinath ' '1']
 ['prof. s. de' '2']
 ['prof. shamik sen' '1']
 ['prof. t.k.sengupta' '1']]
true_name: dr. a. kushari 
 accuracy:0.9             
number of predictions: 50
time: 23.458990812301636

vedio: QLcxx6MJnbA.mp4
unique predicted value counts:
[['dr. a. kushari' '43']
 ['prof. s. dasgupta' '2']
 ['prof. s. de' '2']
 ['prof. t.k.sengupta' '3']]
true_name: dr. a. kushari 
 accuracy:0.86             
number of predictions: 50
time: 17.181333303451538

vedio: YrkbOeF7wFw.mp4
unique predicted value counts:
[['dr. a. kushari' '15']
 ['dr. p. gopinath ' '3']
 ['prof. amit basak ' '1']
 ['prof. debashis ray' '1']
 ['prof. k. p. sinhamahapatra' '1']
 ['prof. s. dasgupta' '1']
 ['prof. s. de' '9']
 ['prof. t.k.sengupta' '19']]
true_name: dr. a. kushari 
 accuracy:0.3             
number of predictions: 50
time: 33.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:176: RuntimeWarning: Mean of empty slice.
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:182: RuntimeWarning: Mean of empty slice.



vedio: vT40FWuvIB8.mp4
unique predicted value counts:
[['prof. k. p. sinhamahapatra' '1']
 ['prof. t.k.sengupta' '10']]
true_name: prof. t.k.sengupta 
 accuracy:0.9090909090909091             
number of predictions: 11
time: 28.696478128433228



for nptel_channel_name Biotechnology


playlist: Introduction to Mechanobiology

vedio: Vnxy8BcCTUc.mp4
unique predicted value counts:
[['dr. p. gopinath ' '4']
 ['prof. shamik sen' '35']
 ['prof. t.k.sengupta' '5']]
true_name: prof. shamik sen 
 accuracy:0.7954545454545454             
number of predictions: 44
time: 99.31154108047485

vedio: Bu07aFJGeyw.mp4
unique predicted value counts:
[['prof. shamik sen' '34']
 ['prof. t.k.sengupta' '2']]
true_name: prof. shamik sen 
 accuracy:0.9444444444444444             
number of predictions: 36
time: 92.98284864425659

vedio: orfE5Q33yto.mp4
unique predicted value counts:
[['prof. shamik sen' '50']]
true_name: prof. shamik sen 
 accuracy:1.0             
number of predictions: 50
time: 54.52878928

In [ ]:
results_dict_list[0]

{'accuracy': 0.9,
 'counts': array([45,  1,  1,  2,  1]),
 'frame_index_list': [16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  73,
  75,
  76,
  77,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  99,
  100,
  102,
  103,
  105,
  106,
  107,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121],
 'names_list': ['dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'dr. a. kushari',
  'prof. s. de',
  'dr. a. kushari',
  'dr. a. kushari',
  'prof. s. de',
  'dr. a. kushari',
  'dr. a. kushari',
  'prof. s. dasgupta',
  'dr. a. kus

time: 6.1 ms


In [ ]:
# prof_name_2_playlist_dict = {'prof. t.k.sengupta': 'Foundation of Scientific Computing', 
#  None: 'Particle Characterization (PG)',
#  'prof. k. p. sinhamahapatra': 'Introduction to Aerodynamics',
#  'dr. a. kushari': 'Jet and Rocket Propulsion',
#  'prof. bhaskar ot': 'Turbomachinery Aerodynamics',
#  'prof. mainak das': 'Bio electricity',
#  'dr. p. gopinath ': 'Biomedical nanotechnology',
#  'prof. debabrata das': 'Industrial Biotechnology',
#  'prof. shamik sen': 'Introduction to Mechanobiology',
#  'dr. ton er': 'Introduction To Proteomics',
#  'prof s. de': 'Advanced Mathematical Techniques in Chemical Engineering',
#  'dr eue or': 'Mass Transfer Operations I',
#  'prof. s. ganguly': 'Microscale Transport Processes',
#  'prof. gargi das epartment': 'Phase Equilibrium Thermodynamics',
#  'prof: padma vankar': 'Advance Analytical Course',
#  'prof. debashis ray ': 'Analytical Chemistry',
#  'prof. s. dasgupta coma': 'BioChemistry I',
#  'prof. a. g. samuelson': 'Introduction to Organometallic Chemistry',
#  'prof. amit basak ': 'Stereochemistry',
#  'prof. deepak khemani': 'Artificial Intelligence Search Methods for problem Solving',
#  'prof. soumya k. ghosh': 'Cloud Computing',
#  'prof. indranil sengupta': 'Hardware Modeling using Verilog',
#  'dr. yogish sabharwal': 'Introduction to parallel Programming in Open MP',
#  'prof. shalabh ': 'Introduction to R Software'}

# playlist_2_prof_name_dict = {v: k for k, v in prof_name_2_playlist_dict.items()}
# save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_playlist_dict.pkl"), prof_name_2_playlist_dict)
# save_as_pickle(os.path.join(pkl_files_dir, "playlist_2_prof_name_dict.pkl"), playlist_2_prof_name_dict)


# folder_name_2_prof_name_dict = {0: 'prof. k. p. sinhamahapatra',
#  1: 'dr. a. kushari',
#  2: 'prof. bhaskar ot',
#  3: 'prof. mainak das',
#  4: 'dr. p. gopinath ',
#  5: 'prof. debabrata das',
#  6: 'prof. shamik sen',
#  7: 'dr. ton er',
#  8: 'prof s. de',
#  9: 'dr eue or',
#  10: 'prof. s. ganguly',
#  11: 'prof. gargi das epartment',
#  12: 'prof: padma vankar',
#  13: 'prof. debashis ray ',
#  14: 'prof. s. dasgupta coma',
#  15: 'prof. a. g. samuelson',
#  16: 'prof. amit basak ',
#  17: 'prof. deepak khemani',
#  18: 'prof. soumya k. ghosh',
#  19: 'prof. indranil sengupta',
#  20: 'prof. shalabh '}

# prof_name_2_folder_name_dict = {v: k for k, v in folder_name_2_prof_name_dict.items()}
# save_as_pickle(os.path.join(pkl_files_dir, "folder_name_2_prof_name_dict.pkl"), folder_name_2_prof_name_dict)
# save_as_pickle(os.path.join(pkl_files_dir, "prof_name_2_folder_name_dict.pkl"), prof_name_2_folder_name_dict)